In [3]:
%pip install opencv-python tensorflow


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

def main():
    try:
        # Load the trained model
        model = load_model('lie_detector_main.h5')
        # Compile model to avoid warnings (use your original loss and metrics)
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
        
        # Define input size
        IMG_HEIGHT, IMG_WIDTH = 128, 128

        def preprocess_frame(frame):
            frame_resized = cv2.resize(frame, (IMG_WIDTH, IMG_HEIGHT))
            frame_normalized = frame_resized / 255.0
            return np.expand_dims(frame_normalized, axis=0)

        # Start webcam - try different indices if 0 doesn't work
        for i in range(3):  # Try up to 3 different camera indices
            cap = cv2.VideoCapture(i)
            if cap.isOpened():
                print(f"Found webcam at index {i}")
                break
            cap.release()
        else:
            print("Error: Could not open any webcam")
            print("Possible solutions:")
            print("1. Make sure your webcam is properly connected")
            print("2. Check if another application is using the webcam")
            print("3. Try different camera indices (0, 1, 2)")
            return

        while True:
            ret, frame = cap.read()
            if not ret:
                print("Error reading frame from webcam")
                break

            # Flip frame horizontally for more intuitive viewing
            frame = cv2.flip(frame, 1)

            # Preprocess and predict
            input_frame = preprocess_frame(frame)
            prediction = model.predict(input_frame, verbose=0)[0][0]
            confidence = prediction if prediction > 0.5 else 1 - prediction
            label = 'Lie' if prediction > 0.5 else 'Truth'
            color = (0, 0, 255) if label == 'Lie' else (0, 255, 0)

            # Display prediction and confidence
            cv2.putText(frame, f'Prediction: {label}', (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
            cv2.putText(frame, f'Confidence: {confidence:.2f}', (10, 70),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2, cv2.LINE_AA)
            cv2.imshow('Lie Detector', frame)

            # Press 'q' to quit
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    except Exception as e:
        print(f"An error occurred: {str(e)}")
    finally:
        if 'cap' in locals() and cap.isOpened():
            cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

: 

In [5]:
import tensorflow as tf
print(tf.__version__)


2.15.0
